In [76]:
import ee
import geemap

ee.Initialize(project="ee-nathanielpaulrobinson")
print(ee.__version__)

1.1.4


In [78]:
# Load Private Assets
asset_path = "projects/ee-nathanielpaulrobinson/assets/NTRI-data/"
ntri_villages = ee.FeatureCollection(asset_path + "NTRI-villages")
ntri_extent = ee.FeatureCollection(asset_path + "NTRI-boundary")
glad_bareground = ee.Image(asset_path + "bare_00N_030E")

# Load Public Assets
mod44b = ee.ImageCollection("MODIS/006/MOD44B")
rap_veg = ee.ImageCollection("projects/rap-data-365417/assets/vegetation-cover-v3")
hansen_gfc = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")
protected_areas = ee.FeatureCollection("WCMC/WDPA/current/polygons")

# Load Geometry
geometry = ee.Geometry.Polygon(
    [
        [
            [-113.07017637550278, 42.88855499739364],
            [-113.07017637550278, 32.857108964921466],
            [-97.33775450050278, 32.857108964921466],
            [-97.33775450050278, 42.88855499739364],
        ]
    ],
    None,
    False,
)

In [3]:
# Village Selection
assessment_village_names = [
    "Eleng'ata Dapash",
    "Engaruka chini",
    "Kakoi",
    "Katikati",
    "Kimana",
    "Kitwai A",
    "Losirwa",
    "Matale A",
    "Nadonjukin",
    "Ngoswak",
    "Olchoroonyokie",
    "Sangaiwe",
]

assessment_villages = ntri_villages.filter(
    ee.Filter.inList("Village", assessment_village_names)
)

In [15]:
# Create Pixel Area Image
land_mask = hansen_gfc.select("datamask").eq(1)
area = ee.Image.pixelArea().multiply(0.0001).updateMask(land_mask).rename("area")

In [13]:
# MOD44B Processing
nonvegetated_band = "Percent_NonVegetated"
pre_start_date = "2010"
pre_end_date = "2015"
post_start_date = "2016"
post_end_date = "2021"

mod44b_nonvegetated = mod44b.select(nonvegetated_band)

pre_nonvegetated = (
    mod44b_nonvegetated.filterDate(pre_start_date, pre_end_date)
    .mean()
    .rename("pre_nonvegetated_mod44")
)
post_nonvegetated = (
    mod44b_nonvegetated.filterDate(post_start_date, post_end_date)
    .mean()
    .rename("post_nonvegetated_mod44")
)

mod44_bgr = pre_nonvegetated.addBands(post_nonvegetated)

In [39]:
# RAP Processing
rap_bare_ground_band = "BGR"
rap_bgr_collection = rap_veg.select(rap_bare_ground_band)
rap_projection = rap_veg.first().projection()

pre_rap_bgr = (
    rap_bgr_collection.filterDate(pre_start_date, pre_end_date)
    .mean()
    .rename("pre_nonvegetated_rap")
)


post_rap_bgr = (
    rap_bgr_collection.filterDate(post_start_date, post_end_date)
    .mean()
    .rename("post_nonvegetated_rap")
)

rap_bgr = pre_rap_bgr.addBands(post_rap_bgr).setDefaultProjection(rap_projection)

In [44]:
# Create Random Points Across S.W. US for Analysis
random_points = ee.FeatureCollection.randomPoints(geometry, 10000)

In [46]:
# Combine MOD44 and RAP Datasets

mod44_projection = mod44b.first().projection()

rap_bgr_mod = rap_bgr.reproject(mod44_projection)

nonvegetated_rap_mod44 = rap_bgr_mod.addBands(mod44_bgr)  # .divide(100).multiply(area)

bare_stats = nonvegetated_rap_mod44.reduceRegions(
    collection=random_points, scale=240, crs="EPSG:4326", reducer=ee.Reducer.mean()
)

task1 = ee.batch.Export.table.toDrive(
    collection=bare_stats,
    description="mod44_rap_stats",
    fileFormat="CSV",
    fileNamePrefix="mod44_rap_stats",
    folder="ntri_bg_data",
)
# task1.start()

In [89]:
# Combine GLAD BGR and MOD44

mod_bgr_2010 = (
    mod44b_nonvegetated.filterDate("2010", "2011")
    .first()
    .divide(100)
    .multiply(area)
    .rename("mod_bgr")
    .addBands(area)
)
glad_bgr = glad_bareground.divide(100).multiply(area).rename("glad_bgr").addBands(area)


village_mod_stats = mod_bgr_2010.reduceRegions(
    collection=assessment_villages,
    scale=250,
    crs="EPSG:4326",
    reducer=ee.Reducer.sum(),
).select(["Vil_Mtaa_N", "area", "mod_bgr"])

village_glad_stats = glad_bgr.reduceRegions(
    collection=assessment_villages,
    scale=30,
    crs="EPSG:4326",
    reducer=ee.Reducer.sum(),
).select(["Vil_Mtaa_N", "area", "glad_bgr"])


# village_filter = ee.Filter.equals(leftField="Vil_Mtaa_N", rightField="Vil_Mtaa_N")

# innerJoin = ee.Join.inner("primary", "secondary")
# table_join = innerJoin.apply(village_mod_stats, village_glad_stats, village_filter)
# table_join

task2 = ee.batch.Export.table.toDrive(
    collection=village_glad_stats,
    description="village_bare_stats_mod",
    fileFormat="CSV",
    fileNamePrefix="village_bare_stats_mod",
    folder="ntri_bg_data",
)
task2.start()
task3 = ee.batch.Export.table.toDrive(
    collection=village_glad_stats,
    description="village_bare_stats_mod",
    fileFormat="CSV",
    fileNamePrefix="village_bare_stats_mod",
    folder="ntri_bg_data",
)
task3.start()